In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [2]:
def plot_image(image):
    plt.imshow(image, cmap="gray", interpolation="nearest")
    plt.axis("off")

def plot_color_image(image):
    plt.imshow(image.astype(np.uint8),interpolation="nearest")
    plt.axis("off")

In [3]:
import tensorflow as tf

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/fashion")

W0602 08:02:23.568693 140685043943232 deprecation.py:323] From <ipython-input-4-98a006d7060d>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0602 08:02:23.570088 140685043943232 deprecation.py:323] From /usr/local/anaconda/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0602 08:02:23.571698 140685043943232 deprecation.py:323] From /usr/local/anaconda/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future ve

Extracting data/fashion/train-images-idx3-ubyte.gz


W0602 08:02:23.860056 140685043943232 deprecation.py:323] From /usr/local/anaconda/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0602 08:02:23.914436 140685043943232 deprecation.py:323] From /usr/local/anaconda/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting data/fashion/train-labels-idx1-ubyte.gz
Extracting data/fashion/t10k-images-idx3-ubyte.gz
Extracting data/fashion/t10k-labels-idx1-ubyte.gz


In [5]:
type(mnist.train)

tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet

In [6]:
n_epochs = 3
batch_size = 100

X_batch, y_batch = mnist.train.next_batch(batch_size)


In [11]:
X_batch.shape

(100, 784)

In [7]:

height=28
width=28
channels=1
n_inputs=height*width

conv1_fmaps=32
conv1_ksize=3
conv1_stride=1
conv1_pad="SAME"

conv2_fmaps=64
conv2_ksize=3
conv2_stride=2
conv2_pad="SAME"

pool3_fmaps=conv2_fmaps

n_fc1=64
n_outputs=10

reset_graph()

with tf.name_scope("inputs"):
    X=tf.placeholder(tf.float32,shape=[None,n_inputs],name="X")
    X_reshaped=tf.reshape(X, shape=[-1,height,width,channels])
    y=tf.placeholder(tf.int32,shape=[None],name="y")

#conv1=tf.layers.conv2d(X_reshaped,filters=32,kernel_size=3,
                      #strides=1,padding="SAME",activation=tf.nn.relu,name="conv1")

    
conv2=tf.layers.conv2d(X_reshaped,filters=64,kernel_size=3,
                      strides=2,padding="SAME",activation=tf.nn.relu,name="conv2")


with tf.name_scope("pool3"):
    pool3=tf.nn.max_pool(conv2,ksize=[1,2,2,1],strides=[1,2,2,1],padding="VALID")
    pool3_flat=tf.reshape(pool3,shape=[-1,pool3_fmaps*7*7])
    
with tf.name_scope("fc1"):
    fc1=tf.layers.dense(pool3_flat,n_fc1,activation=tf.nn.relu,name="fc1")

with tf.name_scope("output"):
    logits=tf.layers.dense(fc1, n_outputs, name="output")
    Y_proba=tf.nn.softmax(logits,name="Y_proba")

with tf.name_scope("train"):
    xentropy=tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss=tf.reduce_mean(xentropy)
    optimizer=tf.train.AdamOptimizer()
    training_op=optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct=tf.nn.in_top_k(logits,y,1)
    accuracy=tf.reduce_mean(tf.cast(correct,tf.float32))
    
with tf.name_scope("init_and_save"):
    init=tf.global_variables_initializer()
    saver=tf.train.Saver()
    


W0602 08:02:24.488409 140685043943232 deprecation.py:323] From <ipython-input-7-a02d615c0a64>:33: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0602 08:02:24.495289 140685043943232 deprecation.py:506] From /usr/local/anaconda/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0602 08:02:24.721601 140685043943232 deprecation.py:323] From <ipython-input-7-a02d615c0a64>:41: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.


In [84]:
pool3

<tf.Tensor 'pool3/MaxPool:0' shape=(?, 6, 6, 64) dtype=float32>

In [82]:
# We can visualize the graph from inside Jupyter.
# Just pass the default graph to show_graph function
# Graph appears in the Iframe inside Jupyter notebook

from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = b"<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
show_graph(tf.get_default_graph())


In [83]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples//batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op,feed_dict={X: X_batch, y: y_batch})
        acc_train=accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test=accuracy.eval(feed_dict={X: mnist.test.images, y:mnist.test.labels})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
        
        save_path=saver.save(sess, "model_ckps/my_fashion_mnist_cnn_model")

InvalidArgumentError: Input to reshape is a tensor with 230400 values, but the requested shape requires a multiple of 3136
	 [[node pool3/Reshape (defined at <ipython-input-80-f930d742160f>:38) ]]

Errors may have originated from an input operation.
Input Source operations connected to node pool3/Reshape:
 pool3/MaxPool (defined at <ipython-input-80-f930d742160f>:37)

Original stack trace for 'pool3/Reshape':
  File "/usr/local/anaconda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/anaconda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/anaconda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/anaconda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    se

In [12]:
mnist.train.num_examples//batch_size

550

In [14]:
X_batch.shape

(100, 784)